In [ ]:
library("tidyverse")
library("GOplot")
library("clusterProfiler")
library("org.Hs.eg.db")

# A. GO enrichment analysis of genes harboring significant AS events

## 1. prepare data

In [ ]:
dir = "results/rMATs/sig"
rmats_df = list()
for (file in dir("results/rMATs/sig")) {
   rmats = read.table(paste0(dir, "/", file), header = T, sep = "\t")
    rmats_df[[file]] = rmats
}
# combine gain and loss file of the same AS type, like A3SS.sig.gain.txt and A3SS.sig.lose.txt
AS_type = c("A3SS", "A5SS", "MXE", "RI", "SE")
for (i in 1:length(AS_type)) {
    rmats_df[[paste0(AS_type[i])]] = rbind(rmats_df[[paste0(AS_type[i], ".sig.gain.txt")]], rmats_df[[paste0(AS_type[i], ".sig.lose.txt")]])
}
# drop elements end with .txt
rmats_df = rmats_df[!grepl(".txt", names(rmats_df))]
gene_list = list()
for (i in 1:length(AS_type)) {
    gene_list[[paste0(AS_type[i])]] = rmats_df[[paste0(AS_type[i])]] %>% dplyr::select(c("GeneID", "geneSymbol","IncLevelDifference"))
    # add new column of AS type
    gene_list[[paste0(AS_type[i])]] = cbind(gene_list[[paste0(AS_type[i])]], AS_type[i])
}
# rbind all AS type
gene_list = do.call(rbind, gene_list)

## 2. clusterProfiler

In [74]:
go_list = list()
for (ont in c("BP", "CC", "MF")) {
    go_list[[ont]] = enrichGO(gene = gene_list$GeneID, 
                              OrgDb = org.Hs.eg.db, 
                              keyType = "ENSEMBL",
                              ont = ont, 
                              pAdjustMethod = "BH", 
                              pvalueCutoff = 0.05, 
                            #   qvalueCutoff = 0.05, 
                              readable = T)
}
go_list

$BP
#
# over-representation test
#
#...@organism 	 Homo sapiens 
#...@ontology 	 BP 
#...@keytype 	 ENSEMBL 
#...@gene 	 chr [1:57] "ENSG00000079134" "ENSG00000114867" "ENSG00000288869" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...0 enriched terms found
#...Citation
 T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu.
 clusterProfiler 4.0: A universal enrichment tool for interpreting omics data.
 The Innovation. 2021, 2(3):100141 


$CC
#
# over-representation test
#
#...@organism 	 Homo sapiens 
#...@ontology 	 CC 
#...@keytype 	 ENSEMBL 
#...@gene 	 chr [1:57] "ENSG00000079134" "ENSG00000114867" "ENSG00000288869" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...0 enriched terms found
#...Citation
 T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu.
 clusterProfiler 4.0: A universal enrichment tool for interpreting omics data.
 The Innovation. 2021, 2(3):100141 


$MF
#


只有MF有两个GO term，其他都没有。可能是数据的原因？

In [ ]:
# genes = A data frame with columns for 'ID', 'logFC' selected from rmat_df
# rename GeneID to ID, IncLevelDifference to logFC in gene_list
gene_list_sel = gene_list %>% dplyr::select(c("GeneID", "IncLevelDifference"))
colnames(gene_list_sel) = c("ID", "logFC")
get_circ = function(terms, gene, onc) {
 circ = circle_dat(terms, gene)
 return(circ)
}
circ_res = pmap(list(terms = go_list, gene = gene_list_sel, onc = c("BP", "CC", "MF")), get_circ)

In [70]:
# gene_list$geneSymbol
go_list

$MF
#
# over-representation test
#
#...@organism 	 Homo sapiens 
#...@ontology 	 MF 
#...@keytype 	 ENSEMBL 
#...@gene 	 chr [1:57] "ENSG00000079134" "ENSG00000114867" "ENSG00000288869" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...2 enriched terms found
'data.frame':	2 obs. of  9 variables:
 $ ID         : chr  "GO:0031369" "GO:0097153"
 $ Description: chr  "translation initiation factor binding" "cysteine-type endopeptidase activity involved in apoptotic process"
 $ GeneRatio  : chr  "3/45" "2/45"
 $ BgRatio    : chr  "33/21059" "13/21059"
 $ pvalue     : num  4.76e-05 3.43e-04
 $ p.adjust   : num  0.0059 0.0213
 $ qvalue     : num  0.00446 0.01607
 $ geneID     : chr  "EIF4G1/EIF3B/TBL2" "CASP10/CASP2"
 $ Count      : int  3 2
#...Citation
 T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu.
 clusterProfiler 4.0: A universal enrichment tool for interpreting omics data.
 The Innovation. 2021, 2(3):100141 



In [ ]:
%%R
# library(GOplot)
# library(tidyverse)
# GO_data = list(
#     BP = read_tsv("results/fig2/GO_Biological_Process_2023.human.enrichr.reports.txt", skip=1),
#     CC = read_tsv("results/fig2/GO_Cellular_Component_2023.human.enrichr.reports.txt", skip=1),
#     MF = read_tsv("results/fig2/GO_Molecular_Function_2023.human.enrichr.reports.txt", skip=1)
# )
circ = circle_dat(GO_data$BP %>% as.data.frame())
GO_data$BP

In [ ]:
# !R -e "install.packages('lazyeval', repos='https://mirrors.tuna.tsinghua.edu.cn/CRAN/')" 
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
GOplot = importr('GOplot')
utils = importr('utils')
base = importr('base')

# check if pyforest is loaded and delete it from modules if it is
if 'pyforest' in sys.modules:
    del sys.modules['pyforest']

# unimport pyforest
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.ipython.ggplot import image_png

In [ ]:
GO_data = utils.read_csv(f"results/fig2/GO_Biological_Process_2023.human.enrichr.reports.txt", sep="\t")
# print(GO_data)
GO_data